In [1]:
import numpy as np
class Conv2D():
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, output_padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros', dtype=None):
        
        self.weights = None
        self.bias_weights = None
        
        self.dtype = dtype
        
        padding_modes = ['zeros', 'replicate']
        if padding_mode not in padding_modes:
            raise ValueError("Invalid padding_mode")
        self.padding_mode = padding_mode
        
        if isinstance(in_channels, int) and in_channels > 0:
            self.in_channels = in_channels
        else:
            raise ValueError("Invalid in_channels")
        
        if isinstance(out_channels, int) and out_channels > 0:
            self.out_channels = out_channels
        else:
            raise ValueError("Invalid out_channels")
        
        if isinstance(groups, int) and groups > 0:
            self.groups = groups
        else:
            raise ValueError("Invalid groups")
        
        if isinstance(bias, int) or isinstance(bias, bool):
            self.bias = bool(bias)
        else:
            raise ValueError("Invalid bias")
        
        if isinstance(kernel_size, tuple):
            try:
                self.kernel1, self.kernel2 = kernel_size
                if not (isinstance(self.kernel1, int) and isinstance(self.kernel2, int)):
                    raise ValueError("Invalid kernel_size types")
            except ValueError:
                raise ValueError("Invalid tuple format for kernel_size")
        elif isinstance(kernel_size, int) and kernel_size > 0:
            self.kernel1 = self.kernel2 = kernel_size
        else:
            raise ValueError("Invalid kernel_size")
        
        if isinstance(stride, tuple):
            try:
                self.stride1, self.stride2 = stride
                if not (isinstance(self.stride1, int) and isinstance(self.stride2, int)):
                    raise ValueError("Invalid stride types")
            except ValueError:
                raise ValueError("Invalid tuple format for stride")
        elif isinstance(stride, int) and stride > 0:
            self.stride1 = self.stride2 = stride
        else:
            raise ValueError("Invalid stride")
        
        if isinstance(padding, str):
            if padding in ["valid", "same"]:
                if padding == 'same' and self.stride1 != 1:
                    raise ValueError("padding 'same' is not valid for stride > 1")
                self.padding1 = self.padding2 = padding
            else:
                raise ValueError("Invalid padding")
        elif isinstance(padding, tuple):
            try:
                self.padding1, self.padding2 = padding
                if not (isinstance(self.padding1, int) and isinstance(self.padding2, int)):
                    raise ValueError("Invalid padding types")
            except ValueError:
                raise ValueError("Invalid tuple format for padding")
        elif isinstance(padding, int) and padding > -1:
            self.padding1 = self.padding2 = padding
        else:
            raise ValueError("Invalid padding")
            
        if isinstance(dilation, tuple):
            try:
                self.dilation1, self.dilation2 = dilation
                if not (isinstance(self.dilation1, int) and isinstance(self.dilation2, int)):
                    raise ValueError("Invalid dilation types")
            except ValueError:
                raise ValueError("Invalid tuple format for dilation")
        elif isinstance(dilation, int) and dilation > 0:
            self.dilation1 = self.dilation2 = dilation
        else:
            raise ValueError("Invalid dilation")
        if not((self.in_channels % self.groups == 0) and (self.out_channels % self.groups == 0)):
            raise ValueError("in_channels and out_channels must both be divisible by groups")
            
    def set_weights(self, weights, bias = None):
        if self.bias == True and (type(bias) == type(np.array([]))):
            if len(bias.shape) == 1 and bias.shape[0] == self.out_channels:
                self.bias_weights = bias
            else:
                raise TypeError("Invalid bias weights shape")
        if self.bias == True and (type(bias) != type(np.array([]))):
            raise TypeError("Invalid bias weights")
        
        if type(weights) != type(np.array([])):
            raise TypeError("Invalid weights")
        if len(weights.shape) != 4:
            raise ValueError("Invalid weights shape")
        if weights.shape[0] != self.out_channels:
            raise ValueError(f"Incorrect axis=0 weights dimension, given {weights.shape[0]}, expected {self.out_channels}")
        if weights.shape[1] != self.in_channels // self.groups:
            raise ValueError(f"Incorrect axis=1 weights dimension, given {weights.shape[1]}, expected {self.in_channels // self.groups}")
        if weights.shape[2] != self.kernel1:
            raise ValueError(f"Incorrect axis=2 weights dimension, given {weights.shape[2]}, expected {self.kernel1}")
        if weights.shape[3] != self.kernel2:
            raise ValueError(f"Incorrect axis=3 weights dimension, given {weights.shape[3]}, expected {self.kernel2}")
        self.weights = weights
        
    
    def __get_conv(self, channels, weights, h_out, w_out, offset=0):
        k1_m = self.kernel1 + (self.kernel1 - 1) * (self.dilation1 - 1)
        k2_m = self.kernel2 + (self.kernel2 - 1) * (self.dilation2 - 1)
        conv = []
        for b, k in enumerate(weights):
            out = np.zeros((h_out, w_out), dtype=self.dtype)
            for i in range(len(channels)):
                # padding partition
                # реализовал два режима: zeros и replicate
                ch = channels[i]
                
                for h in range(self.padding1):
                    if (self.padding_mode == 'zeros'):
                        ch = np.vstack((ch, np.zeros(ch.shape[1])))
                        ch = np.vstack((np.zeros(ch.shape[1]), ch))
                    elif (self.padding_mode == 'replicate'):
                        ch = np.vstack((ch, np.array(ch[-1])))
                        ch = np.vstack((np.array(ch[0]), ch))
                   
                for w in range(self.padding2):
                    if (self.padding_mode == 'zeros'):
                        ch = np.hstack((ch, np.zeros((ch.shape[0], 1))))
                        ch = np.hstack((np.zeros((ch.shape[0], 1)), ch))
                    elif (self.padding_mode == 'replicate'):
                        ch = np.hstack((ch, np.expand_dims(np.array(ch[:, -1]), axis=0).T))
                        ch = np.hstack((np.expand_dims(np.array(ch[:, 0]), axis=0).T, ch))
                
                if ch.shape[0] < k1_m or ch.shape[1] < k2_m:
                    raise RuntimeError(f"Channel shape which is {ch.shape} smaller than calculated kernel {k1_m, k2_m}")
                
                # канал по размерам подогнан, пора считать
                m_ind, n_ind = 0, 0
                x, y = 0, 0
                m, n = out.shape
                while m_ind < m:
                    x = 0
                    n_ind = 0
                    while n_ind < n:
                        out[m_ind, n_ind] += np.sum(ch[y:k1_m+y:self.dilation1, x:k2_m+x:self.dilation2] * k[i])
                        n_ind += 1
                        x += self.stride2
                    y += self.stride1
                    m_ind += 1
            if self.bias:
                out += self.bias_weights[b+offset]
            conv.append(out)
        conv = np.asarray(conv)
        return conv
    
    def forward(self, tensor):
        if len(tensor.shape) == 3:
            tensor = np.expand_dims(tensor, axis=0)
        if len(tensor.shape) != 4:
            raise ValueError(f"Invalid tensor dimensions = {len(tensor.shape)}, expected 3 or 4")
        N, c_in, h_in, w_in = tensor.shape
        k1_m = self.kernel1 + (self.kernel1 - 1) * (self.dilation1 - 1)
        k2_m = self.kernel2 + (self.kernel2 - 1) * (self.dilation2 - 1)
        if self.padding1 == 'valid':
            self.padding1 = self.padding2 = 0
        if self.padding1 == 'same':
            self.padding1 = int((h_in * self.stride1 - self.stride1 - h_in + self.dilation1 * (self.kernel1 - 1) + 1) / 2)
            self.padding2 = int((w_in * self.stride2 - self.stride2 - w_in + self.dilation2 * (self.kernel2 - 1) + 1) / 2)
                
        c_out = self.out_channels
        h_out = int((h_in + 2 * self.padding1 - self.dilation1 * (self.kernel1 - 1) - 1) / self.stride1) + 1
        w_out = int((w_in + 2 * self.padding2 - self.dilation2 * (self.kernel2 - 1) - 1) / self.stride2) + 1
        try:
            out_tensor = np.zeros((N, c_out, h_out, w_out), dtype=self.dtype)
        except:
            raise TypeError("Invalid dtype")
        
        for n in range(N):
            cur_out_channel = 0
            step_for_kernels = self.out_channels // self.groups
            step_for_channels = c_in // self.groups
            ker_pos = 0
            ch_pos = 0
            for i in range(self.groups):
                current_channels = tensor[n, ch_pos:ch_pos+step_for_channels]
                current_weights = self.weights[ker_pos:ker_pos+step_for_kernels]
                conv = self.__get_conv(current_channels, current_weights, h_out, w_out, offset=i*step_for_kernels)
                ker_pos += step_for_kernels
                ch_pos += step_for_channels
                for c in conv:
                    out_tensor[n, cur_out_channel] = c
                    cur_out_channel += 1
        return out_tensor

In [2]:
import torch
from tqdm import tqdm

In [3]:
rng = np.random.default_rng()

In [4]:
def test():
    for i in (pbar := tqdm(range(10))):
        t = rng.integers(low=0, high=255, size=(5, in_channels, 28, 28)) / 255.0
        tt = torch.Tensor(t)
        t_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                 bias=bias, groups=groups, stride=stride,
                                 padding=padding, dilation=dilation, padding_mode=padding_mode)

        # берем случайные веса из слоя torch и записываем их в наш кастомный класс
        weights = t_conv.weight.detach().numpy()
        if bias:
            bias_w = t_conv.bias.detach().numpy()
        layer = Conv2D(in_channels, out_channels, kernel_size=kernel_size,
                       groups=groups, bias=bias, stride=stride, padding=padding,
                       dilation=dilation, padding_mode=padding_mode)
        if bias:
            layer.set_weights(weights, bias_w)
        else:
            layer.set_weights(weights)
        torch_result = t_conv(tt).detach().numpy()
        layer_result = layer.forward(t)
        assert np.allclose(torch_result, layer_result, atol=0.0001), "Error"
    print('pass')

In [5]:
# параметры тестирования для первой группы
kernel_size = (3, 3)
padding = 'same'
padding_mode = 'zeros'
dilation = 1
stride = 1
in_channels = 8
out_channels = 20
bias = True
groups = 2

In [6]:
test()

100%|██████████| 10/10 [00:22<00:00,  2.28s/it]

pass


In [7]:
# параметры тестирования для второй группы
kernel_size = (2, 4)
padding = 'same'
padding_mode = 'replicate'
dilation = 2
stride = 1
in_channels = 4
out_channels = 16
bias = True
groups = 1

In [8]:
test()

100%|██████████| 10/10 [00:18<00:00,  1.84s/it]

pass


In [9]:
# параметры тестирования для третьей группы
kernel_size = (2, 4)
padding = 'valid'
padding_mode = 'replicate'
dilation = 1
stride = (2, 3)
in_channels = 8
out_channels = 4
bias = True
groups = 2

In [10]:
test()

100%|██████████| 10/10 [00:00<00:00, 12.67it/s]

pass


In [11]:
# параметры тестирования для четвертой группы
kernel_size = (3, 2)
padding = 1
padding_mode = 'zeros'
dilation = 2
stride = (4, 3)
in_channels = 5
out_channels = 32
bias = True
groups = 1

In [12]:
test()

100%|██████████| 10/10 [00:04<00:00,  2.32it/s]

pass
